In [9]:
# Convenient jupyter setup
%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [10]:
from src.constants import PAISAGENSLIDAR_PATH

In [27]:
# Find files
las_files = list(PAISAGENSLIDAR_PATH.glob("**/*.las"))
laz_files = list(PAISAGENSLIDAR_PATH.glob("**/*.laz"))
print(f"Found {len(las_files)} las files.\nFound {len(laz_files)} laz files.")

Found 18 las files.
Found 1 laz files.


In [26]:
las_sample = str(las_files[-1])
laz_sample = str(laz_files[0])
print(f"LAS sample: {las_sample}")
print(f"LAZ sample: {laz_sample}")

LAS sample: /gws/nopw/j04/forecol/data/Paisagenslidar/Mission_2015/CON_A01_2015_LiDAR/CON_A01_2015_LAS/CONL4935C83975.las
LAZ sample: /gws/nopw/j04/forecol/data/Paisagenslidar/Mission_2008/TAP_A06_2008_LiDAR/TAP_A06_2008_LiDAR/TAP_A06_2008_laz/TAP_A06_17-01.laz


In [32]:
%%time
%%R -i las_sample
library(lidR)

las <- readLAS(las_sample)
print(las)
las_check(las)
print(las@header)
print(las@proj4string)

class        : LAS (v1.0 format 1)
memory       : 17.7 Mb 
extent       : 493500, 493582.5, 8397500, 8397595 (xmin, xmax, ymin, ymax)
coord. ref.  : NA 
area         : 3950.188 units²
points       : 232.2 thousand points
density      : 58.78 points/units²

 Checking the data
  - Checking coordinates... ✓
  - Checking coordinates type... ✓
  - Checking coordinates quantization... ✓
  - Checking attributes type... ✓
  - Checking ReturnNumber validity... ✓
  - Checking NumberOfReturns validity... ✓
  - Checking ReturnNumber vs. NumberOfReturns... ✓
  - Checking RGB validity... ✓
  - Checking absence of NAs... ✓
  - Checking duplicated points...
   ⚠ 349 points are duplicated and share XYZ coordinates with other points
  - Checking degenerated ground points...
   ⚠ There were 3 degenerated ground points. Some X Y Z coordinates were repeated.
  - Checking attribute population...
   ⚠ 'ScanDirectionFlag' attribute is not populated.
   ⚠ 'EdgeOfFlightline' attribute is not populated.
  - Chec

In [33]:
%%R
mycsf <- csf(sloop_smooth = TRUE, class_threshold = 1, cloth_resolution = 1, time_step = 1)
las <- classify_ground(las, mycsf)
nlas <- normalize_height(las, knnidw())

R[write to console]: Original dataset already contains 7641 ground points. These points were reclassified as 'unclassified' before performing a new ground classification.



In [35]:
%%R
print(nlas)

class        : LAS (v1.0 format 1)
memory       : 20.4 Mb 
extent       : 493500, 493582.5, 8397500, 8397595 (xmin, xmax, ymin, ymax)
coord. ref.  : NA 
area         : 3950.188 units²
points       : 232.2 thousand points
density      : 58.78 points/units²


In [38]:
%%R
las <- classify_ground(las, csf())

R[write to console]: Original dataset already contains 38852 ground points. These points were reclassified as 'unclassified' before performing a new ground classification.



In [39]:
%%R -o bbox
bbox <- las@bbox

In [40]:
bbox

array([[ 493500.01,  493582.5 ],
       [8397500.  , 8397595.4 ]])

In [41]:
%%R
las_check(nlas)


 Checking the data
  - Checking coordinates... ✓
  - Checking coordinates type... ✓
  - Checking coordinates quantization... ✓
  - Checking attributes type... ✓
  - Checking ReturnNumber validity... ✓
  - Checking NumberOfReturns validity... ✓
  - Checking ReturnNumber vs. NumberOfReturns... ✓
  - Checking RGB validity... ✓
  - Checking absence of NAs... ✓
  - Checking duplicated points...
   ⚠ 349 points are duplicated and share XYZ coordinates with other points
  - Checking degenerated ground points...
   ⚠ There were 82 degenerated ground points. Some X Y Z coordinates were repeated.
   ⚠ There were 210 degenerated ground points. Some X Y coordinates were repeated but with different Z coordinates.
  - Checking attribute population...
   ⚠ 'ScanDirectionFlag' attribute is not populated.
   ⚠ 'EdgeOfFlightline' attribute is not populated.
  - Checking gpstime incoherances
    ✗ 2961 pulses (points with the same gpstime) have points with identical ReturnNumber
  - Checking flag attrib